In [1]:
import pandas as pd
import numpy as np

In [2]:
dengue_data = pd.read_csv("data/cleaned_dengue_merged.csv")
dengue_labels = pd.read_csv("data/dengue_labels_train.csv")
dengue_data

,city,year,weekofyear,total_cases,week_start_date,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
0,sj,1990,18,4,1990-04-30,0.122600,0.103725,0.198483,0.177617,12.42,...,32.00,73.365714,12.42,14.012857,2.628571,25.442857,6.900000,29.4,20.0,16.0
1,sj,1990,19,5,1990-05-07,0.169900,0.142175,0.162357,0.155486,22.82,...,17.94,77.368571,22.82,15.372857,2.371429,26.714286,6.371429,31.7,22.2,8.6
2,sj,1990,20,4,1990-05-14,0.032250,0.172967,0.157200,0.170843,34.54,...,26.10,82.052857,34.54,16.848571,2.300000,26.714286,6.485714,32.2,22.8,41.4
3,sj,1990,21,3,1990-05-21,0.128633,0.245067,0.227557,0.235886,15.36,...,13.90,80.337143,15.36,16.672857,2.428571,27.471429,6.771429,33.3,23.3,4.0
4,sj,1990,22,6,1990-05-28,0.196200,0.262200,0.251200,0.247340,7.52,...,12.20,80.460000,7.52,17.210000,3.014286,28.942857,9.371429,35.0,23.9,5.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451,iq,2010,21,5,2010-05-28,0.342750,0.318900,0.256343,0.292514,55.30,...,45.00,88.765714,55.30,18.485714,9.800000,28.633333,11.933333,35.4,22.4,27.0
1452,iq,2010,22,8,2010-06-04,0.160157,0.160371,0.136043,0.225657,86.47,...,207.10,91.600000,86.47,18.070000,7.471429,27.433333,10.500000,34.7,21.7,36.6
1453,iq,2010,23,1,2010-06-11,0.247057,0.146057,0.250357,0.233714,58.94,...,50.60,94.280000,58.94,17.008571,7.500000,24.400000,6.900000,32.2,19.2,7.4
1454,iq,2010,24,1,2010-06-18,0.333914,0.245771,0.278886,0.325486,59.67,...,62.33,94.660000,59.67,16.815714,7.871429,25.433333,8.733333,31.2,21.0,16.0


In [3]:
X_train = np.array(dengue_data)[:,1:]
y_train = np.array(dengue_labels)[:,-1]
X_train[:,3], y_train

(array(['1990-04-30', '1990-05-07', '1990-05-14', ..., '2010-06-11',
        '2010-06-18', '2010-06-25'], dtype=object),
 array([4, 5, 4, ..., 1, 1, 4], dtype=object))

In [4]:
def split_date(date_str):
    year, month, day = map(int, date_str.split('-'))
    return year, month, day

dates = X_train[:, 3]

if type(dates[0]) == str:
    years, months, days = zip(*map(split_date, dates))

    # Replace the date column with the split components
    X_train[:, 3] = years
    X_train = np.insert(X_train, 3, months, axis=1)
    X_train = np.insert(X_train, 4, days, axis=1)

# Now convert the numerical columns to float
numerical_columns = [2, 3, 4]
X_train[:, numerical_columns] = X_train[:, numerical_columns].astype(float)

In [5]:
X_train[:,3], X_train[:,4], X_train[:,5], X_train[:,6]

(array([4.0, 5.0, 5.0, ..., 6.0, 6.0, 6.0], dtype=object),
 array([30.0, 7.0, 14.0, ..., 11.0, 18.0, 25.0], dtype=object),
 array([1990, 1990, 1990, ..., 2010, 2010, 2010], dtype=object),
 array([0.1226, 0.1699, 0.03225, ..., 0.2470571, 0.3339143, 0.2981857],
       dtype=object))

In [6]:
X_train[0,5]

1990

In [8]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

ModuleNotFoundError: No module named 'sklearn'

In [ ]:
class Layer:
    def __init__(self, input_size, output_size, id, output_layer=False):
        self.weights = np.random.randn(output_size, input_size).astype(float)
        self.bias = np.zeros((output_size,1)).astype(float)
        self.is_output = output_layer
        self.id = id
        
    def sigmoid(self, x, deriv=False):
        x = x.astype(float)
        if deriv == False:
            exp_x = np.exp(-np.clip(x, -709, 709))  # Clip x to prevent overflow/underflow
            return 1 / (1 + exp_x)
        else:
            sigmoid_x = self.sigmoid(x)
            return sigmoid_x * (1 - sigmoid_x)

    def softmax(self, x, deriv=False):
        if deriv:
            softmax_probs = self.softmax(x)  # Changed: Call self.softmax(x) instead of softmax(x)
            if np.any(np.isnan(softmax_probs)) or np.any(np.isinf(softmax_probs)):
                raise ValueError("Invalid softmax probabilities: contains NaNs or infinities")

            jacobian = np.diagflat(softmax_probs) - np.dot(softmax_probs, softmax_probs.T)

            return np.sum(jacobian, axis=1, keepdims=True)
        else:
            x -= np.max(x)

            exp_x = np.exp(x)
            softmax_probs = exp_x / np.sum(exp_x)

            return softmax_probs.reshape(-1, 1) if softmax_probs.ndim == 1 else softmax_probs
    
    def relu(self, x, deriv=False):
        if deriv == False:
            return np.maximum(0,x)
        else:
            return np.where(x <= 0, 0, 1)
        
    def forward_prop(self, input):
        self.input = input
        self.z = np.dot(self.weights, self.input) + self.bias
        if self.is_output:
            self.a = self.z  # No activation function for the output layer
        else:
            self.a = self.relu(self.z)
        return self.a


    
    def back_prop(self, output, expected, lr, gradient, clip_value):
        # Clip gradient to a certain range
        #clipped_gradient = np.clip(gradient, -clip_value, clip_value)
        clipped_gradient = gradient
        
        E = clipped_gradient
        if not self.is_output:
            activation_prime = self.relu(self.z,True)
            E = E * activation_prime

        return_data = np.dot(self.weights.T, E)
        dCdW = np.dot(E, self.input.T)

        self.weights -= lr * dCdW.astype(float)
        self.bias -= lr * clipped_gradient.astype(float)  # Note: Here, we are using the clipped gradient as it is also the error for the bias

        return return_data

In [ ]:
class NeuralNetwork:
    def __init__(self):
        self.Layers = [Layer(26,15, 'input'), Layer(15,1, 'output', True)]
    
    def forward_prop(self, input):
        for layer in self.Layers:
            input = layer.forward_prop(input)
        return input
    
    def back_prop(self, output, expected, learning_rate):
        gradient = (2/(len(output)))*(output-expected)
        print("error grad:", gradient)
        
        for layer in reversed(self.Layers):
            gradient = layer.back_prop(output, expected, learning_rate, gradient,1000)
    
    def train(self, inputs, outputs, lr):
        epochs = 10000
        for e in range(epochs):
            for i in range(inputs.shape[0]):
                x = np.array(inputs[i,:]).reshape(-1,1)
                y = np.array(outputs[i,:]).reshape(-1,1)
                given_output = self.forward_prop(x)
                self.back_prop(given_output,y,lr)

    def test_train(self, inputs, outputs, lr, epochs):
        # create group of 100 images
        print("test train")
        
        for e in range(epochs):
            if(e % 50 == 0):
                print("epoch",e)
            groupX = inputs
            groupY = outputs

            indices = np.arange(len(groupX))
            np.random.shuffle(indices)
            groupX, groupY = groupX[indices], groupY[indices]
            for i in range(groupX.shape[0]):
                x = np.array(groupX[i,:]).reshape(-1,1)
                y = np.array(groupY[i]).reshape(-1,1)
                given_output = self.forward_prop(x)
                self.back_prop(given_output,y,lr)
    
    def one_pass(self, x):
        given_output = self.forward_prop(x)
        return given_output

In [ ]:
nn = NeuralNetwork()
nn.test_train(X_train, y_train, 0.1, 10)

test train
epoch 0
error grad: [[24437.374014080717]]
error grad: [[-5.118979202045582e+17]]
error grad: [[8.789578496420385e+17]]
error grad: [[-3.681878329751867e+31]]
error grad: [[2.1363355793615672e+85]]
error grad: [[-6.5610623992333235e+152]]
error grad: [[1.20204144931126e+261]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: 

/var/folders/_6/qfy7c9f97wn3g4k1v_b4dvmh0000gn/T/ipykernel_91897/3728848813.py:59: RuntimeWarning: invalid value encountered in multiply
  E = E * activation_prime


error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]
error grad: [[nan]]


In [ ]:
result = nn.one_pass(X_train[100].reshape(-1,1))

result = normaliserX.denormalize(result)
actual = normaliserY.denormalize(y_train)[100]
print(result, actual)


[[nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
  nan nan nan nan nan nan nan nan]] 13830.0
